In [1]:
import pandas as pd
import numpy as np

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler

In [44]:
churnData= pd.read_csv('files_for_lab/Customer-Churn.csv')

In [37]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [45]:
churnData['TotalCharges'].iloc[488]

' '

In [47]:
def replacing_white(x):
    if x == ' ':
        x = '0'
        return x
    else :return x

In [48]:
churnData['TotalCharges']= churnData['TotalCharges'].apply(replacing_white)

In [49]:
churnData['TotalCharges']= pd.to_numeric(churnData.TotalCharges)

In [50]:
red_churn= churnData[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']]

In [51]:
red_churn

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
0,1,0,29.85,29.85
1,34,0,56.95,1889.50
2,2,0,53.85,108.15
3,45,0,42.30,1840.75
4,2,0,70.70,151.65
...,...,...,...,...
7038,24,0,84.80,1990.50
7039,72,0,103.20,7362.90
7040,11,0,29.60,346.45
7041,4,1,74.40,306.60


In [52]:
scaler = MinMaxScaler()
red_churn_scaled = scaler.fit_transform(red_churn)
red_churn_scaled= pd.DataFrame(red_churn_scaled, columns= red_churn.columns)

In [53]:
red_churn_scaled

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
0,0.013889,0.0,0.115423,0.003437
1,0.472222,0.0,0.385075,0.217564
2,0.027778,0.0,0.354229,0.012453
3,0.625000,0.0,0.239303,0.211951
4,0.027778,0.0,0.521891,0.017462
...,...,...,...,...
7038,0.333333,0.0,0.662189,0.229194
7039,1.000000,0.0,0.845274,0.847792
7040,0.152778,0.0,0.112935,0.039892
7041,0.055556,1.0,0.558706,0.035303


In [54]:
X= red_churn_scaled.drop(columns= 'TotalCharges')
y= red_churn_scaled['TotalCharges']

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y)